# Анализ влияния пищевой промышленности на окружающую среду.

Общеизвестный факт, что в развитых экономиках часто перепроизводятся товары широкого потребления, что приводит к совершенно безрезультатным тратам ресурсов. Однако у этого явления есть не только экономические недостатки (неэффективные траты на закупки лишнего сырья поставки и рекламу невостребованного товара, перегруз рабочих, траты на утилизацию нераскупленных товаров, но и экологические. В своем проекте я по возможности подробно изучу влияние работы пищевой промышленности на мировую экологию и наглядно продемонстрирую его. Этот проект поможет потребителям осознать и их причастность к происходящим экологическим катастрофам по всему миру и подтолкнет их к более осознанному ограниченному потреблению, симулирующему и ограниченное производство. 

## Библиотеки

В этом проекте мне понадобятся библиотеки для скачивания файлов, работы с базами данных и математические библиотеки. 

Установим их:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import urllib.request

## Данные

#### Скачивание

Скачаем необходимые базы данных из открытого проекта "Our World in Data" (https://ourworldindata.org/environmental-impacts-of-food)


In [2]:
land = pd.read_csv('land_per_kilo.csv', delimiter=',')
water = pd.read_csv('water_per_kilo.csv', delimiter=',') 
GHG = pd.read_csv('GHG_emns_per_kilo.csv', delimiter=',')
eutrophication = pd.read_csv('eutrophn_per_kilo.csv', delimiter=',')

Убедимся, что данные скачались - посмотрим на случайные 5 строк каждого датасета:

In [3]:
land.sample(5)

,Entity,Code,Year,"Land use per kilogram (Poore & Nemecek, 2018)"
23,Other Fruit,NaN,2010,0.89
25,Other Vegetables,NaN,2010,0.38
32,Root Vegetables,NaN,2010,0.33
1,Bananas,NaN,2010,1.93
29,Poultry Meat,NaN,2010,12.22


В данных есть пропущенные значения, но если они только в коде продукта, то это не так принципиально для исследования: можем, например, сгруппировать по названию товара и вообще выкинуть коды продуктов.

Позже мы проверим, действительно ли данные пропущенны только там.

In [4]:
water.sample(5)

,Entity,Code,Year,"Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)"
23,Other Fruit,NaN,2010,153.5
36,Wheat & Rye,NaN,2010,647.5
2,Barley,NaN,2010,17.1
5,Beet Sugar,NaN,2010,217.7
26,Peas,NaN,2010,396.6


In [5]:
GHG.sample(5)

,Entity,Code,Year,"GHG emissions per kilogram (Poore & Nemecek, 2018)"
13,Dark Chocolate,NaN,2010,46.65
31,Rice,NaN,2010,4.45
24,Other Pulses,NaN,2010,1.79
6,Berries & Grapes,NaN,2010,1.53
12,Coffee,NaN,2010,28.53


In [6]:
eutrophication.sample(5)

,Entity,Code,Year,"Eutrophying emissions per kilogram (Poore & Nemecek, 2018)"
20,Nuts,NaN,2010,19.15
29,Poultry Meat,NaN,2010,48.70
22,Onions & Leeks,NaN,2010,3.24
17,Lamb & Mutton,NaN,2010,97.13
37,Wine,NaN,2010,4.57


### Проверка на NaN и чистка

Подсчитаем количество строк в датасетах, сравним его с количеством NaN-ов, и если они совпадают, то удалим абсолютно пустой столбец с кодом продукта за ненадобностью.

In [7]:
# Подсчитаем количество строк в датасете
land.count()[0]

38

In [8]:
# Подсчитаем количества NaN-ов в каждом столбце
land.isnull().sum ()

Entity                                            0
Code                                             38
Year                                              0
Land use per kilogram (Poore & Nemecek, 2018)     0
dtype: int64

Действительно, пропущенные значения есть только в столбце с кодом продукта. Удалим этот столбец, чтобы не "таскать за собой" лишний объем памяти.

In [9]:
# Удаляем пустой столбец
del land[land.columns [1]]

# Проверяем результаты удаления
land.sample(5)

# Столбец удален!

,Entity,Year,"Land use per kilogram (Poore & Nemecek, 2018)"
9,Cassava,2010,1.81
22,Onions & Leeks,2010,0.39
1,Bananas,2010,1.93
32,Root Vegetables,2010,0.33
6,Berries & Grapes,2010,2.41


Проведем аналогичные операции с другими используемыми датасетами. Не будем их комментировать, так как они фактически копируют действия выше

In [13]:
# Подсчитаем количество строк в датасете
print(water.count()[0])

# Подсчитаем количества NaN-ов в каждом столбце
water.isnull().sum ()

38


Entity                                                          0
Code                                                           38
Year                                                            0
Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)     0
dtype: int64

In [15]:
# Удаляем пустой столбец
del water[water.columns [1]]

# Проверяем результаты удаления
water.sample(5)

# Столбец удален!

,Entity,Year,"Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)"
18,Maize,2010,215.7
35,Tomatoes,2010,369.8
11,Citrus Fruit,2010,82.7
9,Cassava,2010,0.0
1,Bananas,2010,114.5


In [16]:
# Подсчитаем количество строк в датасете
print(GHG.count()[0])

# Подсчитаем количества NaN-ов в каждом столбце
GHG.isnull().sum ()

38


Entity                                                 0
Code                                                  38
Year                                                   0
GHG emissions per kilogram (Poore & Nemecek, 2018)     0
dtype: int64

In [17]:
# Удаляем пустой столбец
del GHG[GHG.columns [1]]

# Проверяем результаты удаления
GHG.sample(5)

# Столбец удален!

,Entity,Year,"GHG emissions per kilogram (Poore & Nemecek, 2018)"
20,Nuts,2010,0.43
7,Brassicas,2010,0.51
14,Eggs,2010,4.67
12,Coffee,2010,28.53
9,Cassava,2010,1.32


In [18]:
# Подсчитаем количество строк в датасете
print(eutrophication.count()[0])

# Подсчитаем количества NaN-ов в каждом столбце
eutrophication.isnull().sum ()

38


Entity                                                         0
Code                                                          38
Year                                                           0
Eutrophying emissions per kilogram (Poore & Nemecek, 2018)     0
dtype: int64

In [19]:
# Удаляем пустой столбец
del eutrophication[eutrophication.columns [1]]

# Проверяем результаты удаления
eutrophication.sample(5)

# Столбец удален!

,Entity,Year,"Eutrophying emissions per kilogram (Poore & Nemecek, 2018)"
7,Brassicas,2010,5.01
12,Coffee,2010,110.52
0,Apples,2010,1.45
36,Wheat & Rye,2010,7.16
17,Lamb & Mutton,2010,97.13


Все датасеты отчищены от NaN-ов!

Для удобства, соберем их в один датасет (у каждого датасета отличается только последний столбец - влияние производства 1 кг конкретного товара).

In [26]:
# Создадим новый датафрейм
general_impact = land

# 
general_impact[
              'Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)'
              ] = water['Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)']
general_impact[
              'GHG emissions per kilogram (Poore & Nemecek, 2018)'
              ] = GHG['GHG emissions per kilogram (Poore & Nemecek, 2018)']
general_impact[
              'Eutrophying emissions per kilogram (Poore & Nemecek, 2018)'
              ] = eutrophication['Eutrophying emissions per kilogram (Poore & Nemecek, 2018)']

general_impact.sample(5)

,Entity,"Land use per kilogram (Poore & Nemecek, 2018)","Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)","GHG emissions per kilogram (Poore & Nemecek, 2018)","Eutrophying emissions per kilogram (Poore & Nemecek, 2018)"
3,Beef (beef herd),326.21,1451.2,99.48,301.41
26,Peas,7.46,396.6,0.98,7.52
21,Oatmeal,7.60,482.4,2.48,11.23
1,Bananas,1.93,114.5,0.86,3.29
28,Potatoes,0.88,59.1,0.46,3.48


Наконец, мы можем притсупить к анализу полученного датасета!

# Визуализация и анализ упорядоченных данных

Для визуализации нам придётся сгруппировать строки в таблице - иначе у нас будет слишком много столбцов, чтобы что-то разглядеть. 

In [31]:
general_impact.groupby('Entity'). quantile (.99)

,"Land use per kilogram (Poore & Nemecek, 2018)","Freshwater withdrawals per kilogram (Poore & Nemecek, 2018)","GHG emissions per kilogram (Poore & Nemecek, 2018)","Eutrophying emissions per kilogram (Poore & Nemecek, 2018)"
Entity,,,,
Apples,0.63,180.1,0.43,1.45
Bananas,1.93,114.5,0.86,3.29
Barley,1.11,17.1,1.18,2.33
Beef (beef herd),326.21,1451.2,99.48,301.41
Beef (dairy herd),43.24,2714.3,33.30,365.29
Beet Sugar,1.83,217.7,1.81,5.41
Berries & Grapes,2.41,419.6,1.53,6.12
Brassicas,0.55,119.4,0.51,5.01
Cane Sugar,2.04,620.1,3.20,16.92
